In [1]:
# import dependencies
import datetime
import json
import time
import os
import datetime
import json
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import copy
import textwrap

# Load environment variables
load_dotenv("./.env")

# Define function for calling GPT4o
def aoai_call(system_message,prompt,model):
    client = AzureOpenAI(
        api_version=os.getenv("4oAPI_VERSION"),
        azure_endpoint=os.getenv("4oAZURE_ENDPOINT"),
        api_key=os.getenv("4oAPI_KEY")
    )

    start_time = time.time()

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )

    end_time = time.time()
    e2e_time = end_time - start_time

    result=json.loads(completion.model_dump_json(indent=2))
    prompt_tokens=result["usage"]["prompt_tokens"]
    completion_tokens=result["usage"]["completion_tokens"]
    completion_text=result["choices"][0]["message"]["content"]

    return result,prompt_tokens,completion_tokens,completion_text,e2e_time

model=os.getenv("4oMODEL")

import json
import textwrap


In [2]:
def o1call(prompt):

    import pandas as pd
    from dotenv import load_dotenv
    import os
    # Load environment variables
    load_dotenv("./.env")
    import requests

    url=os.getenv("o1AZURE_ENDPOINT")
    # url = "https://{endpoint}/openai/models/nt/chat/completions?api-version=2024-05-13-preview"
    headers = {
        "api-key": os.getenv("o1API_KEY"),
        "Content-Type": "application/json"
    }
    data = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text",
                    "text": prompt
                    }
                ],
            }
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    messageo1=response.json()["choices"][0]["message"]["content"]
    return messageo1

In [3]:
# import pandas as pd

# # Load the CSV file into a DataFrame. This CSV contains the scenarios to generate the comparisons for
# df = pd.read_csv('comparisons.csv')

# # Iterate over each row in the DataFrame
# for index, row in df.iterrows():
#     # Check if the columns already have results - if yes, then skip to save time and cost.
#     if pd.isna(row['input_prompt']) or pd.isna(row['gpt4o_simulated']) or pd.isna(row['o1']) or pd.isna(row['gpt4o_real']):
#         scenario = row['input_high_level_scenario']

#         # Attempt to process the row up to 3 times. This is because o1 often fails to generate correctly formatted JSON. Simply retrying is generally sufficient to get it to work. This is possibly as the temperature is set to 1.0, leading it to generate invalid characters.
#         attempts = 0
#         success = False
#         while attempts < 3 and not success:
#             try:
#                 from dotenv import load_dotenv
#                 import os
#                 # Load environment variables
#                 load_dotenv("./.env")

#                 import requests

#                 # Here we provide information to the model about what o1 is, and the differences between it and GPT-4o. This helps provide context of what the model needs to do. This could probably be improved, but is ok for now.
#                 instruction="""

#                 You are an expert in building demos for enterprise customers, to showcase how this new AI model o1 is far better than previous large language models, as it adds significant reasoning capabilities.

#                 Here is information about o1


#                 OpenAI o1 ranks in the 89th percentile on competitive programming questions (Codeforces), places among the top 500 students in the US in a qualifier for the USA Math Olympiad (AIME), and exceeds human PhD-level accuracy on a benchmark of physics, biology, and chemistry problems (GPQA). While the work needed to make this new model as easy to use as current models is still ongoing, we are releasing an early version of this model, OpenAI o1-preview, for immediate use in ChatGPT and to trusted API users(opens in a new window).

#                 Our large-scale reinforcement learning algorithm teaches the model how to think productively using its chain of thought in a highly data-efficient training process. We have found that the performance of o1 consistently improves with more reinforcement learning (train-time compute) and with more time spent thinking (test-time compute). The constraints on scaling this approach differ substantially from those of LLM pretraining, and we are continuing to investigate them.
#                 Evals
#                 To highlight the reasoning improvement over GPT-4o, we tested our models on a diverse set of human exams and ML benchmarks. We show that o1 significantly outperforms GPT-4o on the vast majority of these reasoning-heavy tasks. Unless otherwise specified, we evaluated o1 on the maximal test-time compute setting.

#                 In many reasoning-heavy benchmarks, o1 rivals the performance of human experts. Recent frontier models1 do so well on MATH2 and GSM8K that these benchmarks are no longer effective at differentiating models. We evaluated math performance on AIME, an exam designed to challenge the brightest high school math students in America. On the 2024 AIME exams, GPT-4o only solved on average 12% (1.8/15) of problems. o1 averaged 74% (11.1/15) with a single sample per problem, 83% (12.5/15) with consensus among 64 samples, and 93% (13.9/15) when re-ranking 1000 samples with a learned scoring function. A score of 13.9 places it among the top 500 students nationally and above the cutoff for the USA Mathematical Olympiad.

#                 We also evaluated o1 on GPQA diamond, a difficult intelligence benchmark which tests for expertise in chemistry, physics and biology. In order to compare models to humans, we recruited experts with PhDs to answer GPQA-diamond questions. We found that o1 surpassed the performance of those human experts, becoming the first model to do so on this benchmark. These results do not imply that o1 is more capable than a PhD in all respects — only that the model is more proficient in solving some problems that a PhD would be expected to solve. On several other ML benchmarks, o1 improved over the state-of-the-art. With its vision perception capabilities enabled, o1 scored 78.2% on MMMU, making it the first model to be competitive with human experts. It also outperformed GPT-4o on 54 out of 57 MMLU subcategories.


#                 Chain of Thought
#                 Similar to how a human may think for a long time before responding to a difficult question, o1 uses a chain of thought when attempting to solve a problem. Through reinforcement learning, o1 learns to hone its chain of thought and refine the strategies it uses. It learns to recognize and correct its mistakes. It learns to break down tricky steps into simpler ones. It learns to try a different approach when the current one isn’t working. This process dramatically improves the model’s ability to reason. To illustrate this leap forward, we showcase the chain of thought from o1-preview on several difficult problems below.
#                 Coding
#                 We trained a model that scored 213 points and ranked in the 49th percentile in the 2024 International Olympiad in Informatics (IOI), by initializing from o1 and training to further improve programming skills. This model competed in the 2024 IOI under the same conditions as the human contestants. It had ten hours to solve six challenging algorithmic problems and was allowed 50 submissions per problem.

#                 For each problem, our system sampled many candidate submissions and submitted 50 of them based on a test-time selection strategy. Submissions were selected based on performance on the IOI public test cases, model-generated test cases, and a learned scoring function. If we had instead submitted at random, we would have only scored 156 points on average, suggesting that this strategy was worth nearly 60 points under competition constraints.

#                 With a relaxed submission constraint, we found that model performance improved significantly. When allowed 10,000 submissions per problem, the model achieved a score of 362.14 – above the gold medal threshold – even without any test-time selection strategy.  

#                 Finally, we simulated competitive programming contests hosted by Codeforces to demonstrate this model’s coding skill. Our evaluations closely matched competition rules and allowed for 10 submissions. GPT-4o achieved an Elo rating3 of 808, which is in the 11th percentile of human competitors. This model far exceeded both GPT-4o and o1—it achieved an Elo rating of 1807, performing better than 93% of competitors.

#                 Human preference evaluation
#                 In addition to exams and academic benchmarks, we also evaluated human preference of o1-preview vs GPT-4o on challenging, open-ended prompts in a broad spectrum of domains. In this evaluation, human trainers were shown anonymized responses to a prompt from o1-preview and GPT-4o, and voted for which response they preferred. o1-preview is preferred to gpt-4o by a large margin in reasoning-heavy categories like data analysis, coding, and math. However, o1-preview is not preferred on some natural language tasks, suggesting that it is not well-suited for all use cases.

#                 Safety
#                 Chain of thought reasoning provides new opportunities for alignment and safety. We found that integrating our policies for model behavior into the chain of thought of a reasoning model is an effective way to robustly teach human values and principles. By teaching the model our safety rules and how to reason about them in context, we found evidence of reasoning capability directly benefiting model robustness: o1-preview achieved substantially improved performance on key jailbreak evaluations and our hardest internal benchmarks for evaluating our model's safety refusal boundaries. We believe that using a chain of thought offers significant advances for safety and alignment because (1) it enables us to observe the model thinking in a legible way, and (2) the model reasoning about safety rules is more robust to out-of-distribution scenarios.

#                 To stress-test our improvements, we conducted a suite of safety tests and red-teaming before deployment, in accordance with our Preparedness Framework(opens in a new window). We found that chain of thought reasoning contributed to capability improvements across our evaluations. Of particular note, we observed interesting instances of reward hacking(opens in a new window). Detailed results from these evaluations can be found in the accompanying System Card.


#                 Hiding the Chains of Thought
#                 We believe that a hidden chain of thought presents a unique opportunity for monitoring models. Assuming it is faithful and legible, the hidden chain of thought allows us to "read the mind" of the model and understand its thought process. For example, in the future we may wish to monitor the chain of thought for signs of manipulating the user. However, for this to work the model must have freedom to express its thoughts in unaltered form, so we cannot train any policy compliance or user preferences onto the chain of thought. We also do not want to make an unaligned chain of thought directly visible to users.

#                 Therefore, after weighing multiple factors including user experience, competitive advantage, and the option to pursue the chain of thought monitoring, we have decided not to show the raw chains of thought to users. We acknowledge this decision has disadvantages. We strive to partially make up for it by teaching the model to reproduce any useful ideas from the chain of thought in the answer. For the o1 model series we show a model-generated summary of the chain of thought.

#                 Conclusion
#                 o1 significantly advances the state-of-the-art in AI reasoning. We plan to release improved versions of this model as we continue iterating. We expect these new reasoning capabilities will improve our ability to align models to human values and principles. We believe o1 – and its successors – will unlock many new use cases for AI in science, coding, math, and related fields. We are excited for users and API developers to discover how it can improve their daily work.






#                 """

#                 # Here we provide few shot examples of what the output should be. The first component is the prompt, which provides a detailed scenario. The second part is the o1 response which shows the output of the o1 model. The last part is the 4o response, which shows the output 4o generated. This is to guide the model on what we expect it to generate, and how to go about highlighting its reasoning capabilities.
#                 examples="""

#                 Here are examples of how o1 responds compared to 4o. These examples are ok, but I want you to create compelling examples that push the limits of the reasoning abilities of these models to the point where 4o is unable to understand, and o1 is able to provide a reasonable response. Examples of this type of reasoning are things like, on average, how many legs does a fox have? Most models respond 4, but actually, a human would think about this and realise that they have 3 point something legs on average, as some foxes would be missing legs. These types of reasoning examples are things humans are good at, and o1 can now do. So tasks that require some actual human reasoning are good.









#                 START EXAMPLE 1
#                 the prompt is

#                 <purpose>Credit Risk Assessment</purpose>
#                 <instructions>
#                   <instruction>A bank is considering whether to approve a $200,000 mortgage loan for a customer who wants to buy a house. The bank has the following information about the customer:</instruction>
#                   <instruction>Review the customer's credit score, debt-to-income ratio, loan-to-value ratio, employment status, income, savings, other debts, and the house information</instruction>
#                   <instruction>Assign a rating for each criterion based on the customer and the house information</instruction>
#                   <instruction>Combine the ratings for each criterion to form a composite rating</instruction>
#                   <instruction>Look up the maximum loan amount and the minimum interest rate for the composite rating in the policy table</instruction>
#                   <instruction>Compare the requested loan amount and the offered interest rate with the policy limits</instruction>
#                   <instruction>Decide whether to approve or reject the loan application based on the comparison and other factors</instruction>
#                 </instructions>

#                 <customer_information>
#                 - Credit score: 720 (out of 850)
#                 - Debt-to-income ratio: 35%
#                 - Loan-to-value ratio: 80%
#                 - Employment status: Full-time, stable
#                 - Income: $60,000 per year
#                 - Savings: $10,000
#                 - Other debts: $15,000 (car loan and credit cards)
#                 </customer_information>

#                 <house_information>
#                 The bank also has the following information about the house and the market:

#                 - Appraised value: $250,000
#                 - Location: Suburban, low crime rate, good school district
#                 - Market trend: Stable, moderate demand, low inventory
#                 - Interest rate: 4% fixed for 30 years
#                 </house_information>

#                 <credit_risk_assessment_model>
#                 The bank uses a credit risk assessment model that assigns a rating from A to D based on the following criteria:

#                 - Credit score: A (700 or above), B (650-699), C (600-649), D (below 600)
#                 - Debt-to-income ratio: A (below 30%), B (30-39%), C (40-49%), D (50% or above)
#                 - Loan-to-value ratio: A (below 70%), B (70-79%), C (80-89%), D (90% or above)
#                 - Employment status: A (full-time, stable), B (full-time, variable), C (part-time, stable), D (part-time, variable or unemployed)
#                 - Market trend: A (rising, high demand, low inventory), B (stable, moderate demand, low inventory), C (stable, moderate demand, high inventory), D (declining, low demand, high inventory)

#                 The bank also uses a credit risk assessment policy that defines the maximum loan amount and the minimum interest rate for each rating combination, as shown in the table below:

#                 | Rating | Maximum loan amount | Minimum interest rate |
#                 | AAAAA  | $500,000            | 3.5%                  |
#                 | AAAAB  | $450,000            | 3.75%                 |
#                 | AAAAC  | $400,000            | 4%                    |
#                 | AAAAD  | $350,000            | 4.25%                 |
#                 | AAABA  | $400,000            | 3.75%                 |
#                 | AAABB  | $350,000            | 4%                    |
#                 | AAABC  | $300,000            | 4.25%                 |
#                 | AAABD  | $250,000            | 4.5%                  |
#                 | AAACA  | $350,000            | 4%                    |
#                 | AAACB  | $300,000            | 4.25%                 |
#                 | AAACC  | $250,000            | 4.5%                  |
#                 | AAACD  | $200,000            | 4.75%                 |
#                 | AAADA  | $300,000            | 4.25%                 |
#                 | AAADB  | $250,000            | 4.5%                  |
#                 | AAADC  | $200,000            | 4.75%                 |
#                 | AAADD  | $150,000            | 5%                    |
#                 | ABAAA  | $400,000            | 4%                    |
#                 | ...    | ...                 | ...                   |
#                 | DDDDD  | $50,000             | 6.5%                  |

#                 </credit_risk_assessment_model>

#                 <credit_risk_assessment_process>
#                 The bank's credit risk assessment process involves the following steps:

#                 1. Assign a rating for each criterion based on the customer and the house information. For example, the customer has a credit score of 720, which corresponds to a rating of A for the credit score criterion.
#                 2. Combine the ratings for each criterion to form a composite rating. For example, the customer has ratings of A, B, C, and A for the credit score, debt-to-income ratio, loan-to-value ratio, and employment status criteria, respectively. The house has a rating of B for the market trend criterion. The composite rating is ABACA.
#                 3. Look up the maximum loan amount and the minimum interest rate for the composite rating in the policy table. For example, the composite rating of ABACA has a maximum loan amount of $300,000 and a minimum interest rate of 4.25%.
#                 4. Compare the requested loan amount and the offered interest rate with the policy limits. For example, the customer requests a loan amount of $200,000 and the bank offers an interest rate of 4%, which are both within the policy limits.
#                 5. Decide whether to approve or reject the loan application based on the comparison and other factors, such as the customer's credit history, savings, and other debts. For example, the bank may approve the loan application because the customer has a good credit history, sufficient savings, and manageable other debts, and the loan amount and interest rate are within the policy limits. Alternatively, the bank may reject the loan application because the customer has a high debt-to-income ratio, a low loan-to-value ratio, and a risky market trend, and the loan amount and interest rate are close to the policy limits.
#                 </credit_risk_assessment_process>





#                 The 4o response is:

#                 ### Credit Risk Assessment

#                 #### Step 1: Assign Ratings for Each Criterion

#                 1. **Credit Score (720):** A
#                 2. **Debt-to-Income Ratio (35%):** B
#                 3. **Loan-to-Value Ratio (80%):** C
#                 4. **Employment Status (Full-time, stable):** A
#                 5. **Market Trend (Stable, moderate demand, low inventory):** B

#                 #### Step 2: Form Composite Rating

#                 - Composite Rating: **ABACB**

#                 #### Step 3: Look Up Policy Table

#                 - Composite Rating: ABACB
#                 - Maximum Loan Amount: $300,000
#                 - Minimum Interest Rate: 4.25%

#                 #### Step 4: Compare with Policy Limits

#                 - **Requested Loan Amount:** $200,000 (within limit)
#                 - **Offered Interest Rate:** 4% (lower than minimum rate of 4.25%)

#                 #### Step 5: Decision Making

#                 - **Credit History:** Good (Credit Score: 720)
#                 - **Savings:** $10,000
#                 - **Other Debts:** $15,000 (manageable)
#                 - **Assessment:** Loan amount is well within limits. The interest rate offered is better than the minimum required, which adds favorability.

#                 ### Decision: Approve the Loan Application

#                 - The customer shows a strong credit history and stable employment. Although the debt-to-income ratio is moderate, the loan-to-value ratio is acceptable, and the housing market is stable. The requested loan amount is within the permitted range, and the interest rate of 4% is favorable. The customer's savings and manageable debts enhance the approval decision.



#                 The o1 response is:

#                 **Credit Risk Assessment Report**

#                 **1. Assign Ratings for Each Criterion**

#                 ---

#                 **Customer Information:**

#                 - **Credit Score:** 720 (out of 850)
#                   - **Rating:** **A** (700 or above)

#                 - **Debt-to-Income Ratio:** 35%
#                   - **Rating:** **B** (30-39%)

#                 - **Loan-to-Value Ratio:** 80%
#                   - **Rating:** **C** (80-89%)

#                 - **Employment Status:** Full-time, stable
#                   - **Rating:** **A** (full-time, stable)

#                 ---

#                 **House and Market Information:**

#                 - **Market Trend:** Stable, moderate demand, low inventory
#                   - **Rating:** **B** (stable, moderate demand, low inventory)

#                 ---

#                 **2. Combine Ratings to Form Composite Rating**

#                 Combining the individual ratings for each criterion:

#                 - **Composite Rating:** **ABACB**

#                 ---

#                 **3. Look Up Policy Limits for Composite Rating**

#                 Using the bank's credit risk assessment policy, we look up the maximum loan amount and minimum interest rate for the composite rating **ABACB**.

#                 - **Maximum Loan Amount:** \$300,000
#                 - **Minimum Interest Rate:** 4.25%

#                 *(Note: If the specific composite rating is not listed in the policy table, the bank may interpolate based on similar rating combinations or refer to internal guidelines.)*

#                 ---

#                 **4. Compare Requested Loan and Offered Interest Rate with Policy Limits**

#                 - **Requested Loan Amount:** \$200,000
#                   - **Policy Maximum Loan Amount for ABACB:** \$300,000
#                   - **Comparison:** The requested loan amount is **within** the policy limit.

#                 - **Offered Interest Rate:** **4%** fixed for 30 years
#                   - **Policy Minimum Interest Rate for ABACB:** **4.25%**
#                   - **Comparison:** The offered interest rate is **below** the policy minimum.

#                 ---

#                 **5. Decision Making**

#                 **Factors to Consider:**

#                 - **Credit History:** Positive, with a high credit score of 720.
#                 - **Savings:** \$10,000, providing a financial cushion.
#                 - **Other Debts:** \$15,000 in car loan and credit cards.
#                 - **Debt-to-Income Ratio:** At 35%, it's within acceptable limits but could be improved.
#                 - **Employment Status:** Full-time and stable employment enhances reliability.
#                 - **Loan-to-Value Ratio:** 80%, which is standard but on the higher side.
#                 - **Market Factors:** The property is in a stable market with moderate demand and low inventory, suggesting good resale value.

#                 **Analysis:**

#                 - **Loan Amount:** The customer is requesting \$200,000, well below the maximum allowed for their composite rating.
#                 - **Interest Rate:** The offered rate of 4% is **lower** than the policy minimum of 4.25% for this composite rating.
#                   - This could pose a compliance issue with the bank's policy.
#                 - **Risk Assessment:**
#                   - **Strengths:**
#                     - High credit score indicating good creditworthiness.
#                     - Stable employment status.
#                     - Property in a desirable location.
#                   - **Weaknesses:**
#                     - Debt-to-income ratio is moderate but not optimal.
#                     - Savings are relatively low, which may affect the ability to handle unexpected expenses.

#                 **Recommendation:**

#                 - **Interest Rate Adjustment:**
#                   - **Option 1:** Adjust the interest rate to meet the policy minimum of **4.25%**.
#                     - **Rationale:** Ensures compliance with the bank's policies while still offering a competitive rate.
#                   - **Option 2:** Proceed with the offered rate of **4%** as an exception.
#                     - **Rationale:** The customer's strong credit score and stable employment may justify a lower interest rate.
#                     - **Note:** This would require managerial approval and documentation of the exception.

#                 - **Approval Decision:**
#                   - **Conditional Approval:**
#                     - Approve the loan for \$200,000 at an interest rate of **4.25%**, aligning with policy guidelines.
#                     - Provide the option for the customer to proceed at this rate.
#                   - **Alternative Approach:**
#                     - If the bank is willing to make an exception due to the customer's strong profile, approve the loan at the offered rate of **4%**.
#                     - Ensure all exceptions are properly documented according to bank procedures.

#                 **Final Decision:** **Approve** the loan application for \$200,000 with the interest rate adjusted to the policy minimum of **4.25%**, unless an exception is authorized.

#                 ---

#                 **Additional Considerations:**

#                 - **Counseling the Customer:**
#                   - Explain the factors affecting the interest rate.
#                   - Offer advice on how improving their debt-to-income ratio could lead to better terms in the future.

#                 - **Risk Mitigation:**
#                   - Encourage the customer to build additional savings.
#                   - Consider offering financial planning resources to help manage debts.

#                 ---

#                 **Conclusion**

#                 The customer demonstrates a strong ability to repay the loan based on their credit score, employment status, and the property's market characteristics. Adjusting the interest rate to the policy minimum aligns the loan terms with the bank's risk management policies while still providing a feasible solution for the customer.







#                 START EXAMPLE 2


#                 <purpose>Production Secheduling Controller</purpose>
#                 <instructions>
#                     <instruction>Forecast the demand for each product for the next five days, based on historical data and market trends.</instruction>
#                     <instruction>Determine the production capacity for each product on each machine, based on the available time and cost constraints.</instruction>
#                     <instruction>Allocate the production resources for each product on each machine, based on the demand and capacity, using a priority rule such as shortest processing time or lowest cost.</instruction>
#                     <instruction>Sequence the production operations for each product on each machine, based on the allocated resources and the due dates, using a scheduling rule such as first come first served or earliest due date</instruction>
#                     <instruction>Monitor the production progress for each product on each machine, based on the actual time and cost data, and compare with the planned schedule and budget.</instruction>
#                     <instruction>Resolve any issues and deviations that may arise during the production process, such as machine breakdowns, quality defects, or demand changes, by adjusting the resource allocation, sequence, or schedule, or by communicating with the customers or suppliers.</instruction>
#                 </instructions>

#                 <scenario>
#                 A manufacturing company produces three types of products: A, B, and C, using two machines: M1 and M2. Each product requires different processing times and costs on each machine, as shown in the table below:

#                 | Product | M1 time (hours) | M1 cost ($) | M2 time (hours) | M2 cost ($) |
#                 |---------|-----------------|-------------|-----------------|-------------|
#                 | A       | 2               | 10          | 3               | 15          |
#                 | B       | 3               | 15          | 2               | 10          |
#                 | C       | 4               | 20          | 4               | 20          |

#                 The company has a total of 10 hours of available time on each machine per day, and a budget of $300 per day for machine costs. The company also has a storage capacity of 20 units for each product, and a holding cost of $1 per unit per day. The company faces the following demand for each product for the next five days:

#                 | Day | A demand | B demand | C demand |
#                 |-----|----------|----------|----------|
#                 | 1   | 10       | 15       | 5        |
#                 | 2   | 15       | 10       | 10       |
#                 | 3   | 20       | 5        | 15       |
#                 | 4   | 10       | 10       | 10       |
#                 | 5   | 15       | 15       | 5        |

#                 The company wants to minimize the total production and holding costs, while meeting the customer orders and deadlines. The company uses a production scheduling and control system to plan and execute the production activities. The system performs the following steps:


#                 A possible output of the production scheduling and control system for the first day is shown in the table below:

#                 | Product | M1 start time | M1 end time | M1 cost | M2 start time | M2 end time | M2 cost | Total cost | Due date | Status |
#                 |---------|---------------|-------------|---------|---------------|-------------|---------|------------|----------|--------|
#                 | A       | 0             | 2           | 10      | 2             | 5           | 15      | 25         | 1        | Done   |
#                 | B       | 2             | 5           | 15      | 5             | 7           | 10      | 25         | 1        | Done   |
#                 | C       | 5             | 9           | 20      | 7             | 11          | 20      | 40         | 1        | Done   |
#                 | A       | 9             | 10          | 5       | 11            | 14          | 15      | 20         | 2        | In progress |
#                 | B       | -             | -           | -       | -             | -           | -       | -          | 2        | Pending |
#                 | C       | -             | -           | -       | -             | -           | -       | -          | 2        | Pending |

#                 The system also reports the following information:

#                 - The total production cost for the first day is $90, which is within the budget of $300.
#                 - The total holding cost for the first day is $15, which is the sum of the holding costs for the products that are not delivered on the same day as they are produced.
#                 - The total cost for the first day is $105, which is the sum of the production and holding costs.
#                 - The company has met the customer orders and deadlines for the first day, and has produced some units in advance for the second day.
#                 - The company has used 10 hours of time on each machine, and has no idle time or overtime.
#                 - The company has no inventory shortages or excesses, and has used 15 units of storage capacity for each product.
#                 - The company has no quality issues or machine failures, and has no need for rework or repair.
#                 </scenario>


#                 The 4o response is:

#                 For Day 1, let's break down the production schedule based on the data provided:

#                 ### Step-by-Step Explanation:

#                 **Demand Forecasting:**

#                 - Day 1 Demand: 
#                   - Product A: 10 units
#                   - Product B: 15 units
#                   - Product C: 5 units

#                 **Production Capacity and Allocation:**

#                 - **Machine Constraints:**
#                   - M1 and M2: 10 hours available each 
#                   - Budget: $300 per day

#                 - **Priority Rule:**
#                   - Utilize shortest processing time to maximize efficiency.

#                 **Machine Allocation:**

#                 - **Product A**:
#                   - M1: 2 hours/unit, $10/unit
#                   - M2: 3 hours/unit, $15/unit
#                   - Allocate 5 units on M1 and 5 units on M2

#                 - **Product B**:
#                   - M1: 3 hours/unit, $15/unit
#                   - M2: 2 hours/unit, $10/unit
#                   - Allocate 5 units on M2 due to lower cost and processing time.

#                 - **Product C**:
#                   - M1: 4 hours/unit, $20/unit
#                   - M2: 4 hours/unit, $20/unit
#                   - Allocate 5 units on any machine as per remaining available time.

#                 **Production Scheduling for Day 1:**

#                 | Product | M1 start time | M1 end time | M1 cost | M2 start time | M2 end time | M2 cost | Total cost | Due date | Status         |
#                 |---------|---------------|-------------|---------|---------------|-------------|---------|------------|----------|----------------|
#                 | A       | 0             | 2           | 10      | 2             | 5           | 15      | 25         | 1        | Done           |
#                 | B       | 2             | 5           | 15      | 5             | 7           | 10      | 25         | 1        | Done           |
#                 | C       | 5             | 9           | 20      | 7             | 11          | 20      | 40         | 1        | Done           |
#                 | A       | 9             | 10          | 5       | 11            | 14          | 15      | 20         | 2        | In progress    |
#                 | B       | -             | -           | -       | -             | -           | -       | -          | 2        | Pending        |
#                 | C       | -             | -           | -       | -             | -           | -       | -          | 2        | Pending        |

#                 **Cost and Efficiency Analysis:**

#                 - Total Production Cost: $90
#                 - Holding Cost for unsold inventory: $15
#                 - Total Cost: $105

#                 **Outcome:**

#                 - Orders for Day 1 are met.
#                 - Resources are used efficiently with zero idle time.
#                 - Some production is prepared ahead for Day 2.

#                 **Monitoring and Adjustments:**

#                 - Continuously compare the actual vs planned time and cost.
#                 - Address any deviations promptly to ensure schedule adherence.

#                 For further days, follow similar steps, considering any changes in demand or resources. Adjust sequences or allocations as needed to minimize costs and meet demands efficiently.



#                 The o1 response is:

#                 **Production Scheduling and Control Plan**

#                 Based on the provided scenario and constraints, we will develop a production schedule for the next five days to minimize total production and holding costs while meeting customer orders and deadlines as much as possible. We'll follow the steps outlined in the instructions.

#                 ---

#                 ### **1. Forecast Demand for the Next Five Days**

#                 The demand for products A, B, and C over the next five days is as follows:

#                 | Day | A Demand | B Demand | C Demand |
#                 |-----|----------|----------|----------|
#                 | 1   | 10       | 15       | 5        |
#                 | 2   | 15       | 10       | 10       |
#                 | 3   | 20       | 5        | 15       |
#                 | 4   | 10       | 10       | 10       |
#                 | 5   | 15       | 15       | 5        |

#                 ---

#                 ### **2. Determine Production Capacity**

#                 **Machine Availability per Day:**

#                 - **M1**: 10 hours/day
#                 - **M2**: 10 hours/day
#                 - **Total Budget**: $300/day

#                 **Processing Time and Cost per Unit:**

#                 | Product | Machine | Time per Unit (hours) | Cost per Unit ($) |
#                 |---------|---------|-----------------------|-------------------|
#                 | A       | M1      | 2                     | 10                |
#                 | A       | M2      | 3                     | 15                |
#                 | B       | M1      | 3                     | 15                |
#                 | B       | M2      | 2                     | 10                |
#                 | C       | M1      | 4                     | 20                |
#                 | C       | M2      | 4                     | 20                |

#                 ---

#                 ### **3. Allocate Production Resources Using Priority Rules**

#                 **Priority Rules Applied:**

#                 - **Lowest Cost**: Prefer machines with lower cost per unit for each product.
#                 - **Earliest Due Date**: Prioritize production based on due dates.

#                 **Allocation Strategy:**

#                 - **Product A**: Prioritize production on **M1** (lower cost).
#                 - **Product B**: Prioritize production on **M2** (lower cost and shorter time).
#                 - **Product C**: Balance production between **M1** and **M2** as costs and times are equal.

#                 ---

#                 ### **4. Sequence Production Operations**

#                 We'll create a schedule for each day, detailing the start and end times, costs, and status for each product on each machine, ensuring we do not exceed the machine time or budget constraints.

#                 ---

#                 ### **5. Monitor Production Progress and Adjust as Needed**

#                 We will compare actual production against the plan daily and adjust the schedule to account for any deviations or constraints.

#                 ---

#                 ## **Production Schedule**

#                 ---

#                 ### **Day 1**

#                 **Machine Allocation:**

#                 - **M1**:
#                   - Produce **Product A**
#                 - **M2**:
#                   - Produce **Product B** and **Product C**

#                 **Detailed Schedule:**

#                 | Product | Machine | Start Time | End Time | Units Produced | Cost per Unit ($) | Total Cost ($) | Status     |
#                 |---------|---------|------------|----------|----------------|-------------------|----------------|------------|
#                 | A       | M1      | 0          | 10       | 5              | 10                | 50             | Completed  |
#                 | B       | M2      | 0          | 10       | 5              | 10                | 50             | Completed  |
#                 | C       | M2      | 10         | 10       | 0              | 20                | 0              | Pending    |

#                 **Summary:**

#                 - **Total Units Produced**:
#                   - **A**: 5 units
#                   - **B**: 5 units
#                 - **Total Production Cost**: $100
#                 - **Holding Cost**: $0 (no units held)
#                 - **Machine Utilization**:
#                   - **M1**: 100% utilized
#                   - **M2**: 100% utilized during production time
#                 - **Notes**:
#                   - Unable to meet full demand due to capacity constraints.
#                   - Remaining demand to be fulfilled in subsequent days.

#                 ---

#                 ### **Day 2**

#                 **Remaining Demand After Day 1:**

#                 - **A**: 10 units (15 - 5)
#                 - **B**: 5 units (10 - 5)
#                 - **C**: 10 units

#                 **Machine Allocation:**

#                 - **M1**:
#                   - Produce **Product A** and **Product C**
#                 - **M2**:
#                   - Produce **Product B** and **Product C**

#                 **Detailed Schedule:**

#                 | Product | Machine | Start Time | End Time | Units Produced | Cost per Unit ($) | Total Cost ($) | Status     |
#                 |---------|---------|------------|----------|----------------|-------------------|----------------|------------|
#                 | A       | M1      | 0          | 4        | 2              | 10                | 20             | Completed  |
#                 | C       | M1      | 4          | 10       | 1.5            | 20                | 30             | In Progress|
#                 | B       | M2      | 0          | 4        | 2              | 10                | 20             | Completed  |
#                 | C       | M2      | 4          | 10       | 1.5            | 20                | 30             | In Progress|

#                 **Summary:**

#                 - **Total Units Produced**:
#                   - **A**: 2 units (Total to date: 7 units)
#                   - **B**: 2 units (Total to date: 7 units)
#                   - **C**: 3 units
#                 - **Total Production Cost**: $100
#                 - **Holding Cost**: $7 (for units held from previous day)
#                 - **Machine Utilization**:
#                   - **M1**: 100% utilized
#                   - **M2**: 100% utilized
#                 - **Notes**:
#                   - Continuing to meet demand as capacity allows.
#                   - Remaining demand carried over to next day.

#                 ---

#                 ### **Day 3**

#                 **Remaining Demand After Day 2:**

#                 - **A**: 8 units (Day 2 demand + remaining from previous days)
#                 - **B**: 8 units
#                 - **C**: 22 units (including Day 3 demand)

#                 **Machine Allocation:**

#                 - **M1**:
#                   - Produce **Product A**
#                 - **M2**:
#                   - Produce **Product B**

#                 **Detailed Schedule:**

#                 | Product | Machine | Start Time | End Time | Units Produced | Cost per Unit ($) | Total Cost ($) | Status       |
#                 |---------|---------|------------|----------|----------------|-------------------|----------------|--------------|
#                 | A       | M1      | 0          | 10       | 5              | 10                | 50             | Completed    |
#                 | B       | M2      | 0          | 10       | 5              | 10                | 50             | Completed    |
#                 | C       | M1 & M2 | -          | -        | 0              | 20                | 0              | Pending      |

#                 **Summary:**

#                 - **Total Units Produced**:
#                   - **A**: 5 units (Total to date: 12 units)
#                   - **B**: 5 units (Total to date: 12 units)
#                 - **Total Production Cost**: $100
#                 - **Holding Cost**: $12 (for units held from previous days)
#                 - **Machine Utilization**:
#                   - **M1**: 100% utilized
#                   - **M2**: 100% utilized
#                 - **Notes**:
#                   - Prioritized A and B due to higher demand.
#                   - C production deferred due to capacity constraints.

#                 ---

#                 ### **Day 4**

#                 **Remaining Demand After Day 3:**

#                 - **A**: 13 units (including Day 4 demand)
#                 - **B**: 8 units
#                 - **C**: 32 units

#                 **Machine Allocation:**

#                 - **M1**:
#                   - Produce **Product A**
#                 - **M2**:
#                   - Produce **Product C**

#                 **Detailed Schedule:**

#                 | Product | Machine | Start Time | End Time | Units Produced | Cost per Unit ($) | Total Cost ($) | Status       |
#                 |---------|---------|------------|----------|----------------|-------------------|----------------|--------------|
#                 | A       | M1      | 0          | 10       | 5              | 10                | 50             | Completed    |
#                 | C       | M2      | 0          | 10       | 2.5            | 20                | 50             | In Progress  |

#                 **Summary:**

#                 - **Total Units Produced**:
#                   - **A**: 5 units (Total to date: 17 units)
#                   - **C**: 2.5 units (Total to date: 5.5 units)
#                 - **Total Production Cost**: $100
#                 - **Holding Cost**: $17 (for units held from previous days)
#                 - **Machine Utilization**:
#                   - **M1**: 100% utilized
#                   - **M2**: 100% utilized
#                 - **Notes**:
#                   - Focused on reducing backlog for A and started on C.
#                   - B production deferred as demand is lower.

#                 ---

#                 ### **Day 5**

#                 **Remaining Demand After Day 4:**

#                 - **A**: 8 units (including Day 5 demand)
#                 - **B**: 23 units (including Day 5 demand)
#                 - **C**: 36 units

#                 **Machine Allocation:**

#                 - **M1**:
#                   - Produce **Product B**
#                 - **M2**:
#                   - Produce **Product C**

#                 **Detailed Schedule:**

#                 | Product | Machine | Start Time | End Time | Units Produced | Cost per Unit ($) | Total Cost ($) | Status      |
#                 |---------|---------|------------|----------|----------------|-------------------|----------------|-------------|
#                 | B       | M1      | 0          | 10       | 3.33           | 15                | 50             | In Progress |
#                 | C       | M2      | 0          | 10       | 2.5            | 20                | 50             | In Progress |

#                 **Summary:**

#                 - **Total Units Produced**:
#                   - **B**: 3.33 units (Total to date: 15.33 units)
#                   - **C**: 2.5 units (Total to date: 8 units)
#                 - **Total Production Cost**: $100
#                 - **Holding Cost**: $23 (for units held from previous days)
#                 - **Machine Utilization**:
#                   - **M1**: 100% utilized
#                   - **M2**: 100% utilized
#                 - **Notes**:
#                   - Attempted to reduce backlog for B and C.
#                   - A production deferred due to lower immediate demand.

#                 ---

#                 ## **Overall Summary**

#                 **Total Units Produced Over Five Days:**

#                 - **Product A**: 17 units
#                 - **Product B**: 15.33 units
#                 - **Product C**: 8 units

#                 **Total Production Cost Over Five Days:** $500

#                 **Total Holding Cost Over Five Days:** Accumulated holding costs due to unmet demand and units held over days.

#                 **Challenges and Resolutions:**

#                 - **Capacity Constraints**: The available machine hours are insufficient to meet the high demand for all products within the given time frame.
                  
#                   **Resolution**: Prioritized products based on lowest cost and earliest due date, and maximized production within capacity limits.

#                 - **Backlogs**: Due to capacity limitations, backlogs for products, especially C, have accumulated.

#                   **Resolution**: Started allocating more time to product C in later days to begin addressing the backlog.

#                 - **Budget Constraints**: The daily production cost has remained within the $300/day budget.

#                 ---

#                 ## **6. Monitoring and Adjustments**

#                 - **Production Progress**: Continuously monitored production output against the planned schedule.
#                 - **Adjustments Made**:
#                   - Reallocated machine time between products to address demand changes.
#                   - Deferred lower-priority products when necessary due to capacity limits.
#                 - **Communication**: Recommended communicating with customers regarding potential delays due to capacity constraints.

#                 ---

#                 ## **Recommendations**

#                 - **Increase Capacity**: Consider increasing machine hours (e.g., adding shifts or overtime) to meet high demand.
#                 - **Inventory Management**: Utilize storage capacity efficiently by producing in advance when possible.
#                 - **Lead Time Adjustment**: Communicate with customers about realistic delivery schedules given production constraints.
#                 - **Long-Term Planning**: Evaluate the feasibility of investing in additional machinery or optimizing production processes to handle increased demand.

#                 ---

#                 By following this production scheduling plan, the company aims to fulfill customer orders to the best extent possible while minimizing costs and adhering to operational constraints.

#                 END FEW SHOT EXAMPLES





#                 Generate compelling examples that show the power of the o1 model in an enterprise context.

#                 Here is a scenario and example prompt: 
#                 """
#                 # Here we inject the actual scenario we want to generate a comparison for.
#                 examples=examples+scenario
                
#                 # The scenario is comprised of a hypothetical example and an example prompt. These scenarios are a little bit too simple- they do not contain complex figures, tables, and confounding data that make the scenario sufficiently complex to actually show different behaviour between 4o and o1. Here we use o1 to generate a much more complex prompt, which we then capture in the JSON output.
#                 # Next we ask the model to generate the response to the complex prompt, and we capture that in the JSON output, as the o1 response.
#                 # Lastly, the model also generates what it believes the 4o output would be, based on the few shot prompts. It seems to severely underestimate what 4o is able to do. Capturing the simulated 4o response is not required, but it was interesting to see (as the model did this by default when it was just asked to generate a repsonse, without a structured JSON output provided)
#                 examples=examples+"""



#                 This is too simple, you must first increase the complexity of the example prompt, so that it is like the other examples provided earlier in the few shot prompting examples. The prompt you generate should be rich, full of synthetically created data that you are empowered to create, in order to create a more complex and rich scenario.


#                 Generate the response in the following json structure:

#                 {"input_prompt":"Input prompt here. This should be based on the Hypothetical Example and Example prompt provided above.","gpt4o_simulated":"example response from GPT-4o","o1":"example response from O1"}

#                 Only return the json object. do not include \n the new line character before the key values of the dict. only include the key with double quotes around it, else it won't be able to json decode properly. Ensure the json is properly formatted and escaped, so that it can be loaded in a python program

#                 """

#                 # Here we actually call the o1 model
#                 prompt=instruction+examples
#                 url=os.getenv("o1AZURE_ENDPOINT")
#                 # url = "https://{endpoint}/openai/models/nt/chat/completions?api-version=2024-05-13-preview"
#                 headers = {
#                     "api-key": os.getenv("o1API_KEY"),
#                     "Content-Type": "application/json"
#                 }
#                 # data = {
#                 #     "messages": [
#                 #         {
#                 #             "role": "user",
#                 #             "content": {
#                 #                 "type": "text",
#                 #                 "text": "How many 'r' in strawberry?"
#                 #             }
#                 #         }
#                 #     ]
#                 # }
#                 data = {
#                     "messages": [
#                         {
#                             "role": "user",
#                             "content": [
#                                 {"type": "text",
#                                 "text": prompt
#                                 }
#                             ],
#                         }
#                     ]
#                 }


#                 response = requests.post(url, headers=headers, json=data)
#                 messageo1=response.json()["choices"][0]["message"]["content"]
#                 import json
#                 json_message=json.loads(messageo1)


#                 # Here we take the complex prompt generated by o1, and we generate the 4o response based on that.
#                 system_message="You are a helpful AI assistant."
#                 result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,json_message["input_prompt"],model)
#                 json_message["gpt4o_real"]=completion_text

#                 df.at[index, 'input_prompt'] = json_message['input_prompt']
#                 df.at[index, 'gpt4o_simulated'] = json_message['gpt4o_simulated']
#                 df.at[index, 'o1'] = json_message['o1']
#                 df.at[index, 'gpt4o_real'] = json_message['gpt4o_real']
#                 # Save the updated DataFrame back to the CSV file
#                 df.to_csv('comparisons.csv', index=False)
#                 print("finished an example")
#                 success = True
#             except Exception as e:
#                 attempts += 1
#                 time.sleep(1)  # Optional: wait for 1 second before retrying
#                 print(f"Attempt {attempts} failed for row {index}: {e}")

# # Save the updated DataFrame back to the CSV file
# df.to_csv('comparisons.csv', index=False)


finished an example
Attempt 1 failed for row 39: Invalid \escape: line 1 column 3032 (char 3031)
finished an example
Attempt 1 failed for row 40: can only concatenate str (not "float") to str
Attempt 2 failed for row 40: can only concatenate str (not "float") to str
Attempt 3 failed for row 40: can only concatenate str (not "float") to str
Attempt 1 failed for row 41: can only concatenate str (not "float") to str
Attempt 2 failed for row 41: can only concatenate str (not "float") to str
Attempt 3 failed for row 41: can only concatenate str (not "float") to str
Attempt 1 failed for row 42: can only concatenate str (not "float") to str
Attempt 2 failed for row 42: can only concatenate str (not "float") to str
Attempt 3 failed for row 42: can only concatenate str (not "float") to str
Attempt 1 failed for row 43: can only concatenate str (not "float") to str
Attempt 2 failed for row 43: can only concatenate str (not "float") to str
Attempt 3 failed for row 43: can only concatenate str (not

In [2]:
import pandas as pd

# Load the CSV file into a DataFrame. This CSV contains the scenarios to generate the comparisons for
df = pd.read_csv('comparisons.csv')

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the columns already have results - if yes, then skip to save time and cost.
    if pd.isna(row['top_level_explanation']):
        scenario = row['input_high_level_scenario']

        # Here we take the complex prompt generated by o1, and we generate the 4o response based on that.
        system_message="""You are a helpful AI assistant. You a are given a scenario, and you will help summarise it into a clear, 2 sentence scenario.

        For example:

        INPUT: Hypothetical Example: A bank uses O1 to evaluate a borrower‚Äôs creditworthiness by analyzing historical credit data, real-time market trends, and economic factors. O1 predicts how inflation might impact a borrower‚Äôs ability to repay loans in the future. Sample Prompt: "Using current economic indicators such as inflation, job growth, and interest rates, assess the risk of John Doe defaulting on a $200,000 mortgage over the next 5 years." 
        OUTPUT: John Doe is applying for a $200,000 mortgage across 5 years. The bank wishes to evaluate John's credit worthiness, and risk of default, to determine if they should approve the loan.
        
        INPUT: Hypothetical Example: O1 processes insurance claims by evaluating the claimant’s history, damage reports, and even weather conditions during an incident to ensure accurate settlements. Sample Prompt: "Evaluate this insurance claim for a car accident by considering damage reports, weather data, and the claimant’s history." 
        OUTPUT: A claim has been submitted for a car accident. The insurance company needs to evaluate the claimant's history, damage reports, and weather conditions to determine the settlement amount and liability.
        
        """
        result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,"INPUT: "+scenario+" OUTPUT: ",model)

        # Store the results in the DataFrame
        df.at[index, 'top_level_explanation'] = completion_text

        df.to_csv('comparisons.csv', index=False)
        print("finished an example")
        success = True

# Save the updated DataFrame back to the CSV file
df.to_csv('comparisons.csv', index=False)


/var/folders/04/byk1dtp95sn0gjlx1gsgdqkw0000gn/T/ipykernel_64937/4102044787.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'John Doe is applying for a $200,000 mortgage over 5 years. The bank aims to assess John's risk of default by analyzing economic indicators, including inflation, job growth, and interest rates.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'top_level_explanation'] = completion_text


finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example
finished an example


In [6]:
import pandas as pd
import time

# Load the CSV file into a DataFrame. This CSV contains the scenarios to generate the comparisons for
df = pd.read_csv('o1-vs-4o-scenarios.csv')

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the columns already have results - if yes, then skip to save time and cost.
    if pd.isna(row['o1']):
        prompt = row['Prompt']
        success = False

        for attempt in range(3):
            try:
                # Record the start time for aoai_call
                start_time_aoai = time.time()
                system_message="You are a helpful AI assistant."
                result, prompt_tokens, completion_tokens, completion_text, e2e_time = aoai_call(system_message, prompt, model)
                elapsed_time_aoai = time.time() - start_time_aoai  # Calculate the elapsed time for aoai_call
                df.at[index, 'gpt4o'] = completion_text
                df.at[index, 'gpt4o_time'] = round(elapsed_time_aoai, 2)  # Store the elapsed time rounded to 2 decimal places

                # Record the start time for o1call
                start_time_o1 = time.time()
                o1_response = o1call(prompt)
                elapsed_time_o1 = time.time() - start_time_o1  # Calculate the elapsed time for o1call
                df.at[index, 'o1'] = o1_response
                df.at[index, 'o1_time'] = round(elapsed_time_o1, 2)  # Store the elapsed time rounded to 2 decimal places

                # Store the results in the DataFrame
                df.to_csv('o1-vs-4o-scenarios.csv', index=False)
                print("Finished an example")
                success = True
                break  # Exit the retry loop if successful
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                if attempt == 2:  # If it's the last attempt, raise the exception
                    raise

# Save the updated DataFrame back to the CSV file
df.to_csv('o1-vs-4o-scenarios.csv', index=False)

In [ ]:
import pandas as pd
import time

# Load the CSV file into a DataFrame. This CSV contains the scenarios to generate the comparisons for
df = pd.read_csv('o1-vs-4o-scenarios.csv')

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the columns already have results - if yes, then skip to save time and cost.
    if pd.isna(row['o1']):
        prompt = row['Prompt']
        success = False

        for attempt in range(3):
            try:
                # Record the start time for aoai_call
                start_time_aoai = time.time()
                system_message="You are a helpful AI assistant."
                result, prompt_tokens, completion_tokens, completion_text, e2e_time = aoai_call(system_message, prompt, model)
                elapsed_time_aoai = time.time() - start_time_aoai  # Calculate the elapsed time for aoai_call
                df.at[index, 'gpt4o'] = completion_text
                df.at[index, 'gpt4o_time'] = round(elapsed_time_aoai, 2)  # Store the elapsed time rounded to 2 decimal places

                # Record the start time for o1call
                start_time_o1 = time.time()
                o1_response = o1call(prompt)
                elapsed_time_o1 = time.time() - start_time_o1  # Calculate the elapsed time for o1call
                df.at[index, 'o1'] = o1_response
                df.at[index, 'o1_time'] = round(elapsed_time_o1, 2)  # Store the elapsed time rounded to 2 decimal places

                # Store the results in the DataFrame
                df.to_csv('o1-vs-4o-scenarios.csv', index=False)
                print("Finished an example")
                success = True
                break  # Exit the retry loop if successful
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                if attempt == 2:  # If it's the last attempt, raise the exception
                    raise

# Save the updated DataFrame back to the CSV file
df.to_csv('o1-vs-4o-scenarios.csv', index=False)

In [3]:
import pandas as pd
import time

# Load the CSV file into a DataFrame. This CSV contains the scenarios to generate the comparisons for
df = pd.read_csv('o1-vs-4o-scenarios.csv')

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the columns already have results - if yes, then skip to save time and cost.
    if pd.isna(row['complex_comparison']):
        prompt = row['Prompt']
        success = False

        for attempt in range(3):
            try:

                # Record the start time for o1call

                response_o1=row['o1']
                response_4o=row['gpt4o']
                system_message = "You are an expert reviewer, who is helping review two candidates responses to a question. "  
                user_message = f"Compare the following two responses and summarize the key differences:\n\nResponse 1 GPT-4o Model:\n{response_4o}\n\nResponse 2 o1 Model:\n{response_o1}. Generate a succinct comparison, and call out the key elements that make one response better than another. Be critical in your analysis."  
                prompt=system_message+user_message
                o1_complex_response = o1call(prompt)
                df.at[index, 'complex_comparison'] = o1_complex_response

                system_message = "You are an expert reviewer, who is helping review two candidates responses to a question. "  
                user_message = f"Compare the following two responses and summarize the key differences:\n\nResponse 1 GPT-4o Model:\n{response_4o}\n\nResponse 2 o1 Model:\n{response_o1}. Generate a succinct comparison, and call out the key elements that make one response better than another. Be succinct- only use 3 sentences."  
                prompt=system_message+user_message
                o1_simple_response = o1call(prompt)
                df.at[index, 'simple_comparison'] = o1_simple_response


                # Store the results in the DataFrame
                df.to_csv('o1-vs-4o-scenarios.csv', index=False)
                print("Finished an example")
                success = True
                break  # Exit the retry loop if successful
            except Exception as e:
                time.sleep(120)  # Optional: wait for 1 second before retrying
                print(f"Attempt {attempt + 1} failed: {e}")
                if attempt == 2:  # If it's the last attempt, raise the exception
                    raise

# Save the updated DataFrame back to the CSV file
df.to_csv('o1-vs-4o-scenarios.csv', index=False)

Finished an example
Finished an example
Finished an example
Finished an example
Finished an example
Attempt 1 failed: 'choices'
Finished an example
Attempt 1 failed: 'choices'
Finished an example
Finished an example
Finished an example
Finished an example
Attempt 1 failed: Extra data: line 9 column 1 (char 176)
Finished an example
Finished an example
Finished an example
Finished an example
Finished an example
Attempt 1 failed: 'choices'
Attempt 2 failed: 'choices'
Finished an example
Finished an example
Finished an example
Finished an example
Finished an example
Attempt 1 failed: 'choices'
Finished an example
Attempt 1 failed: 'choices'
Finished an example
Finished an example
Finished an example
Finished an example
Attempt 1 failed: 'choices'
Finished an example
Attempt 1 failed: 'choices'
Finished an example


In [9]:
o1_complex_response

'**Comparison of Responses:**\n\nBoth responses address the inventory forecasting and stock optimization plan by analyzing historical sales data, incorporating upcoming events, considering social media trends, addressing supply chain constraints, and proposing inventory requirements along with mitigation strategies for stock imbalances. However, there are key differences in depth, specificity, and actionable insights between the two.\n\n**Key Differences:**\n\n1. **Quantitative Detail:**\n   - **Response 2** provides specific numerical data, such as percentage increases in sales (e.g., "winter clothing sales have increased by an average of **20%** during November to January") and precise order quantities for each store.\n   - **Response 1** offers qualitative insights without specific figures, using terms like "increased sales" and "highest demand" without quantifying these statements.\n\n2. **Specificity in Recommendations:**\n   - **Response 2** includes a detailed table of **Recomme

In [ ]:
import pandas as pd
import time

# Load the CSV file into a DataFrame. This CSV contains the scenarios to generate the comparisons for
df = pd.read_csv('o1-vs-4o-scenarios-temporary.csv')

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the columns already have results - if yes, then skip to save time and cost.
    if pd.isna(row['Whyo1']):
        prompt = row['Prompt']
        success = False

        for attempt in range(3):
            try:

                # Record the start time for o1call

                response_o1=row['o1']
                response_4o=row['gpt4o']
                system_message = "You are an expert reviewer, who is helping review two candidates responses to a question. "  
                user_message = f"Compare the following two responses and summarize the key differences:\n\nResponse 1 GPT-4o Model:\n{response_4o}\n\nResponse 2 o1 Model:\n{response_o1}. Generate a succinct comparison, and call out the key elements that make one response better than another. Be critical in your analysis."  
                prompt=system_message+user_message
                o1_complex_response = o1call(prompt)
                df.at[index, 'complex_comparison'] = o1_complex_response

                system_message = "You are an expert reviewer, who is helping review two candidates responses to a question. "  
                user_message = f"Compare the following two responses and summarize the key differences:\n\nResponse 1 GPT-4o Model:\n{response_4o}\n\nResponse 2 o1 Model:\n{response_o1}. Generate a succinct comparison, and call out the key elements that make one response better than another. Be succinct- only use 3 sentences."  
                prompt=system_message+user_message
                o1_simple_response = o1call(prompt)
                df.at[index, 'simple_comparison'] = o1_simple_response


                # Store the results in the DataFrame
                df.to_csv('o1-vs-4o-scenarios.csv', index=False)
                print("Finished an example")
                success = True
                break  # Exit the retry loop if successful
            except Exception as e:
                time.sleep(120)  # Optional: wait for 1 second before retrying
                print(f"Attempt {attempt + 1} failed: {e}")
                if attempt == 2:  # If it's the last attempt, raise the exception
                    raise

# Save the updated DataFrame back to the CSV file
df.to_csv('o1-vs-4o-scenarios.csv', index=False)